In [1]:
  
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(imgs, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            #if epoch <= 19000:
                #if epoch % sample_interval == 0:
                    #self.sample_images(epoch)
            #else:
                #self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN Images\mnist_%d.png" % epoch)
        plt.close()

In [4]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    #model.predict(...)
    wgan = WGAN()
    wgan.train(epochs=20000, batch_size=32, sample_interval=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_______________________________________

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999920] [G loss: 1.000164]


c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.999929] [G loss: 1.000162]
2 [D loss: 0.999930] [G loss: 1.000167]
3 [D loss: 0.999930] [G loss: 1.000162]
4 [D loss: 0.999925] [G loss: 1.000166]
5 [D loss: 0.999925] [G loss: 1.000163]
6 [D loss: 0.999934] [G loss: 1.000165]
7 [D loss: 0.999934] [G loss: 1.000168]
8 [D loss: 0.999931] [G loss: 1.000173]
9 [D loss: 0.999938] [G loss: 1.000172]
10 [D loss: 0.999931] [G loss: 1.000162]
11 [D loss: 0.999939] [G loss: 1.000160]
12 [D loss: 0.999933] [G loss: 1.000161]
13 [D loss: 0.999938] [G loss: 1.000149]
14 [D loss: 0.999948] [G loss: 1.000138]
15 [D loss: 0.999936] [G loss: 1.000119]
16 [D loss: 0.999938] [G loss: 1.000127]
17 [D loss: 0.999936] [G loss: 1.000113]
18 [D loss: 0.999945] [G loss: 1.000124]
19 [D loss: 0.999941] [G loss: 1.000118]
20 [D loss: 0.999941] [G loss: 1.000113]
21 [D loss: 0.999944] [G loss: 1.000122]
22 [D loss: 0.999947] [G loss: 1.000127]
23 [D loss: 0.999951] [G loss: 1.000107]
24 [D loss: 0.999957] [G loss: 1.000112]
25 [D loss: 0.999950] [G 

...

19124 [D loss: 0.999979] [G loss: 1.000065]
19125 [D loss: 0.999974] [G loss: 1.000063]
19126 [D loss: 0.999968] [G loss: 1.000077]
19127 [D loss: 0.999972] [G loss: 1.000059]
19128 [D loss: 0.999968] [G loss: 1.000061]
19129 [D loss: 0.999969] [G loss: 1.000074]
19130 [D loss: 0.999969] [G loss: 1.000063]
19131 [D loss: 0.999967] [G loss: 1.000066]
19132 [D loss: 0.999968] [G loss: 1.000070]
19133 [D loss: 0.999970] [G loss: 1.000064]
19134 [D loss: 0.999973] [G loss: 1.000067]
19135 [D loss: 0.999971] [G loss: 1.000060]
19136 [D loss: 0.999971] [G loss: 1.000067]
19137 [D loss: 0.999973] [G loss: 1.000066]
19138 [D loss: 0.999965] [G loss: 1.000068]
19139 [D loss: 0.999970] [G loss: 1.000067]
19140 [D loss: 0.999972] [G loss: 1.000061]
19141 [D loss: 0.999972] [G loss: 1.000066]
19142 [D loss: 0.999972] [G loss: 1.000052]
19143 [D loss: 0.999971] [G loss: 1.000060]
19144 [D loss: 0.999963] [G loss: 1.000074]
19145 [D loss: 0.999963] [G loss: 1.000070]
19146 [D loss: 0.999970] [G loss

19311 [D loss: 0.999968] [G loss: 1.000064]
19312 [D loss: 0.999967] [G loss: 1.000074]
19313 [D loss: 0.999969] [G loss: 1.000062]
19314 [D loss: 0.999961] [G loss: 1.000062]
19315 [D loss: 0.999973] [G loss: 1.000048]
19316 [D loss: 0.999963] [G loss: 1.000059]
19317 [D loss: 0.999963] [G loss: 1.000062]
19318 [D loss: 0.999969] [G loss: 1.000073]
19319 [D loss: 0.999973] [G loss: 1.000064]
19320 [D loss: 0.999969] [G loss: 1.000067]
19321 [D loss: 0.999963] [G loss: 1.000064]
19322 [D loss: 0.999970] [G loss: 1.000074]
19323 [D loss: 0.999967] [G loss: 1.000073]
19324 [D loss: 0.999967] [G loss: 1.000064]
19325 [D loss: 0.999966] [G loss: 1.000069]
19326 [D loss: 0.999971] [G loss: 1.000063]
19327 [D loss: 0.999969] [G loss: 1.000059]
19328 [D loss: 0.999965] [G loss: 1.000059]
19329 [D loss: 0.999969] [G loss: 1.000069]
19330 [D loss: 0.999971] [G loss: 1.000068]
19331 [D loss: 0.999967] [G loss: 1.000063]
19332 [D loss: 0.999968] [G loss: 1.000072]
19333 [D loss: 0.999967] [G loss

19498 [D loss: 0.999970] [G loss: 1.000057]
19499 [D loss: 0.999973] [G loss: 1.000063]
19500 [D loss: 0.999965] [G loss: 1.000064]
19501 [D loss: 0.999982] [G loss: 1.000061]
19502 [D loss: 0.999971] [G loss: 1.000066]
19503 [D loss: 0.999970] [G loss: 1.000067]
19504 [D loss: 0.999968] [G loss: 1.000065]
19505 [D loss: 0.999968] [G loss: 1.000067]
19506 [D loss: 0.999969] [G loss: 1.000055]
19507 [D loss: 0.999964] [G loss: 1.000068]
19508 [D loss: 0.999972] [G loss: 1.000068]
19509 [D loss: 0.999969] [G loss: 1.000069]
19510 [D loss: 0.999976] [G loss: 1.000062]
19511 [D loss: 0.999970] [G loss: 1.000054]
19512 [D loss: 0.999969] [G loss: 1.000060]
19513 [D loss: 0.999965] [G loss: 1.000067]
19514 [D loss: 0.999972] [G loss: 1.000059]
19515 [D loss: 0.999969] [G loss: 1.000066]
19516 [D loss: 0.999971] [G loss: 1.000067]
19517 [D loss: 0.999971] [G loss: 1.000064]
19518 [D loss: 0.999971] [G loss: 1.000058]
19519 [D loss: 0.999970] [G loss: 1.000058]
19520 [D loss: 0.999974] [G loss

19685 [D loss: 0.999970] [G loss: 1.000058]
19686 [D loss: 0.999970] [G loss: 1.000062]
19687 [D loss: 0.999970] [G loss: 1.000065]
19688 [D loss: 0.999961] [G loss: 1.000062]
19689 [D loss: 0.999968] [G loss: 1.000064]
19690 [D loss: 0.999972] [G loss: 1.000062]
19691 [D loss: 0.999976] [G loss: 1.000064]
19692 [D loss: 0.999974] [G loss: 1.000069]
19693 [D loss: 0.999976] [G loss: 1.000061]
19694 [D loss: 0.999968] [G loss: 1.000059]
19695 [D loss: 0.999963] [G loss: 1.000060]
19696 [D loss: 0.999966] [G loss: 1.000077]
19697 [D loss: 0.999968] [G loss: 1.000059]
19698 [D loss: 0.999966] [G loss: 1.000065]
19699 [D loss: 0.999973] [G loss: 1.000058]
19700 [D loss: 0.999964] [G loss: 1.000063]
19701 [D loss: 0.999967] [G loss: 1.000066]
19702 [D loss: 0.999964] [G loss: 1.000062]
19703 [D loss: 0.999963] [G loss: 1.000068]
19704 [D loss: 0.999966] [G loss: 1.000061]
19705 [D loss: 0.999965] [G loss: 1.000069]
19706 [D loss: 0.999969] [G loss: 1.000073]
19707 [D loss: 0.999968] [G loss

19872 [D loss: 0.999976] [G loss: 1.000061]
19873 [D loss: 0.999970] [G loss: 1.000058]
19874 [D loss: 0.999971] [G loss: 1.000055]
19875 [D loss: 0.999966] [G loss: 1.000070]
19876 [D loss: 0.999974] [G loss: 1.000055]
19877 [D loss: 0.999961] [G loss: 1.000071]
19878 [D loss: 0.999964] [G loss: 1.000056]
19879 [D loss: 0.999971] [G loss: 1.000074]
19880 [D loss: 0.999971] [G loss: 1.000070]
19881 [D loss: 0.999966] [G loss: 1.000062]
19882 [D loss: 0.999966] [G loss: 1.000068]
19883 [D loss: 0.999971] [G loss: 1.000065]
19884 [D loss: 0.999975] [G loss: 1.000067]
19885 [D loss: 0.999968] [G loss: 1.000066]
19886 [D loss: 0.999966] [G loss: 1.000068]
19887 [D loss: 0.999968] [G loss: 1.000070]
19888 [D loss: 0.999972] [G loss: 1.000073]
19889 [D loss: 0.999969] [G loss: 1.000061]
19890 [D loss: 0.999973] [G loss: 1.000062]
19891 [D loss: 0.999967] [G loss: 1.000058]
19892 [D loss: 0.999969] [G loss: 1.000067]
19893 [D loss: 0.999968] [G loss: 1.000063]
19894 [D loss: 0.999973] [G loss

In [5]:
def generate_images(cnt):
    c=1
    while (c<=cnt):
        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = wgan.generator.predict(noise)
        gen_imgs = 0.5*gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\WGAN Images\generated_wgan\wgan_{}.jpg".format(c), bbox_inches = 'tight', pad_inches = -0.0005)
        print("Images {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)

Images 1 saved.
Images 2 saved.
Images 3 saved.
Images 4 saved.
Images 5 saved.
Images 6 saved.
Images 7 saved.
Images 8 saved.
Images 9 saved.
Images 10 saved.
Images 11 saved.
Images 12 saved.
Images 13 saved.
Images 14 saved.
Images 15 saved.
Images 16 saved.
Images 17 saved.
Images 18 saved.
Images 19 saved.
Images 20 saved.
Images 21 saved.
Images 22 saved.
Images 23 saved.
Images 24 saved.
Images 25 saved.
Images 26 saved.
Images 27 saved.
Images 28 saved.
Images 29 saved.
Images 30 saved.
Images 31 saved.
Images 32 saved.
Images 33 saved.
Images 34 saved.
Images 35 saved.
Images 36 saved.
Images 37 saved.
Images 38 saved.
Images 39 saved.
Images 40 saved.
Images 41 saved.
Images 42 saved.
Images 43 saved.
Images 44 saved.
Images 45 saved.
Images 46 saved.
Images 47 saved.
Images 48 saved.
Images 49 saved.
Images 50 saved.
Images 51 saved.
Images 52 saved.
Images 53 saved.
Images 54 saved.
Images 55 saved.
Images 56 saved.
Images 57 saved.
Images 58 saved.
Images 59 saved.
Images

Images 465 saved.
Images 466 saved.
Images 467 saved.
Images 468 saved.
Images 469 saved.
Images 470 saved.
Images 471 saved.
Images 472 saved.
Images 473 saved.
Images 474 saved.
Images 475 saved.
Images 476 saved.
Images 477 saved.
Images 478 saved.
Images 479 saved.
Images 480 saved.
Images 481 saved.
Images 482 saved.
Images 483 saved.
Images 484 saved.
Images 485 saved.
Images 486 saved.
Images 487 saved.
Images 488 saved.
Images 489 saved.
Images 490 saved.
Images 491 saved.
Images 492 saved.
Images 493 saved.
Images 494 saved.
Images 495 saved.
Images 496 saved.
Images 497 saved.
Images 498 saved.
Images 499 saved.
Images 500 saved.
Images 501 saved.
Images 502 saved.
Images 503 saved.
Images 504 saved.
Images 505 saved.
Images 506 saved.
Images 507 saved.
Images 508 saved.
Images 509 saved.
Images 510 saved.
Images 511 saved.
Images 512 saved.
Images 513 saved.
Images 514 saved.
Images 515 saved.
Images 516 saved.
Images 517 saved.
Images 518 saved.
Images 519 saved.
Images 520

Images 923 saved.
Images 924 saved.
Images 925 saved.
Images 926 saved.
Images 927 saved.
Images 928 saved.
Images 929 saved.
Images 930 saved.
Images 931 saved.
Images 932 saved.
Images 933 saved.
Images 934 saved.
Images 935 saved.
Images 936 saved.
Images 937 saved.
Images 938 saved.
Images 939 saved.
Images 940 saved.
Images 941 saved.
Images 942 saved.
Images 943 saved.
Images 944 saved.
Images 945 saved.
Images 946 saved.
Images 947 saved.
Images 948 saved.
Images 949 saved.
Images 950 saved.
Images 951 saved.
Images 952 saved.
Images 953 saved.
Images 954 saved.
Images 955 saved.
Images 956 saved.
Images 957 saved.
Images 958 saved.
Images 959 saved.
Images 960 saved.
Images 961 saved.
Images 962 saved.
Images 963 saved.
Images 964 saved.
Images 965 saved.
Images 966 saved.
Images 967 saved.
Images 968 saved.
Images 969 saved.
Images 970 saved.
Images 971 saved.
Images 972 saved.
Images 973 saved.
Images 974 saved.
Images 975 saved.
Images 976 saved.
Images 977 saved.
Images 978

Images 1363 saved.
Images 1364 saved.
Images 1365 saved.
Images 1366 saved.
Images 1367 saved.
Images 1368 saved.
Images 1369 saved.
Images 1370 saved.
Images 1371 saved.
Images 1372 saved.
Images 1373 saved.
Images 1374 saved.
Images 1375 saved.
Images 1376 saved.
Images 1377 saved.
Images 1378 saved.
Images 1379 saved.
Images 1380 saved.
Images 1381 saved.
Images 1382 saved.
Images 1383 saved.
Images 1384 saved.
Images 1385 saved.
Images 1386 saved.
Images 1387 saved.
Images 1388 saved.
Images 1389 saved.
Images 1390 saved.
Images 1391 saved.
Images 1392 saved.
Images 1393 saved.
Images 1394 saved.
Images 1395 saved.
Images 1396 saved.
Images 1397 saved.
Images 1398 saved.
Images 1399 saved.
Images 1400 saved.
Images 1401 saved.
Images 1402 saved.
Images 1403 saved.
Images 1404 saved.
Images 1405 saved.
Images 1406 saved.
Images 1407 saved.
Images 1408 saved.
Images 1409 saved.
Images 1410 saved.
Images 1411 saved.
Images 1412 saved.
Images 1413 saved.
Images 1414 saved.
Images 1415 

Images 1796 saved.
Images 1797 saved.
Images 1798 saved.
Images 1799 saved.
Images 1800 saved.
Images 1801 saved.
Images 1802 saved.
Images 1803 saved.
Images 1804 saved.
Images 1805 saved.
Images 1806 saved.
Images 1807 saved.
Images 1808 saved.
Images 1809 saved.
Images 1810 saved.
Images 1811 saved.
Images 1812 saved.
Images 1813 saved.
Images 1814 saved.
Images 1815 saved.
Images 1816 saved.
Images 1817 saved.
Images 1818 saved.
Images 1819 saved.
Images 1820 saved.
Images 1821 saved.
Images 1822 saved.
Images 1823 saved.
Images 1824 saved.
Images 1825 saved.
Images 1826 saved.
Images 1827 saved.
Images 1828 saved.
Images 1829 saved.
Images 1830 saved.
Images 1831 saved.
Images 1832 saved.
Images 1833 saved.
Images 1834 saved.
Images 1835 saved.
Images 1836 saved.
Images 1837 saved.
Images 1838 saved.
Images 1839 saved.
Images 1840 saved.
Images 1841 saved.
Images 1842 saved.
Images 1843 saved.
Images 1844 saved.
Images 1845 saved.
Images 1846 saved.
Images 1847 saved.
Images 1848 

...

Images 7895 saved.
Images 7896 saved.
Images 7897 saved.
Images 7898 saved.
Images 7899 saved.
Images 7900 saved.
Images 7901 saved.
Images 7902 saved.
Images 7903 saved.
Images 7904 saved.
Images 7905 saved.
Images 7906 saved.
Images 7907 saved.
Images 7908 saved.
Images 7909 saved.
Images 7910 saved.
Images 7911 saved.
Images 7912 saved.
Images 7913 saved.
Images 7914 saved.
Images 7915 saved.
Images 7916 saved.
Images 7917 saved.
Images 7918 saved.
Images 7919 saved.
Images 7920 saved.
Images 7921 saved.
Images 7922 saved.
Images 7923 saved.
Images 7924 saved.
Images 7925 saved.
Images 7926 saved.
Images 7927 saved.
Images 7928 saved.
Images 7929 saved.
Images 7930 saved.
Images 7931 saved.
Images 7932 saved.
Images 7933 saved.
Images 7934 saved.
Images 7935 saved.
Images 7936 saved.
Images 7937 saved.
Images 7938 saved.
Images 7939 saved.
Images 7940 saved.
Images 7941 saved.
Images 7942 saved.
Images 7943 saved.
Images 7944 saved.
Images 7945 saved.
Images 7946 saved.
Images 7947 

Images 8329 saved.
Images 8330 saved.
Images 8331 saved.
Images 8332 saved.
Images 8333 saved.
Images 8334 saved.
Images 8335 saved.
Images 8336 saved.
Images 8337 saved.
Images 8338 saved.
Images 8339 saved.
Images 8340 saved.
Images 8341 saved.
Images 8342 saved.
Images 8343 saved.
Images 8344 saved.
Images 8345 saved.
Images 8346 saved.
Images 8347 saved.
Images 8348 saved.
Images 8349 saved.
Images 8350 saved.
Images 8351 saved.
Images 8352 saved.
Images 8353 saved.
Images 8354 saved.
Images 8355 saved.
Images 8356 saved.
Images 8357 saved.
Images 8358 saved.
Images 8359 saved.
Images 8360 saved.
Images 8361 saved.
Images 8362 saved.
Images 8363 saved.
Images 8364 saved.
Images 8365 saved.
Images 8366 saved.
Images 8367 saved.
Images 8368 saved.
Images 8369 saved.
Images 8370 saved.
Images 8371 saved.
Images 8372 saved.
Images 8373 saved.
Images 8374 saved.
Images 8375 saved.
Images 8376 saved.
Images 8377 saved.
Images 8378 saved.
Images 8379 saved.
Images 8380 saved.
Images 8381 

Images 8761 saved.
Images 8762 saved.
Images 8763 saved.
Images 8764 saved.
Images 8765 saved.
Images 8766 saved.
Images 8767 saved.
Images 8768 saved.
Images 8769 saved.
Images 8770 saved.
Images 8771 saved.
Images 8772 saved.
Images 8773 saved.
Images 8774 saved.
Images 8775 saved.
Images 8776 saved.
Images 8777 saved.
Images 8778 saved.
Images 8779 saved.
Images 8780 saved.
Images 8781 saved.
Images 8782 saved.
Images 8783 saved.
Images 8784 saved.
Images 8785 saved.
Images 8786 saved.
Images 8787 saved.
Images 8788 saved.
Images 8789 saved.
Images 8790 saved.
Images 8791 saved.
Images 8792 saved.
Images 8793 saved.
Images 8794 saved.
Images 8795 saved.
Images 8796 saved.
Images 8797 saved.
Images 8798 saved.
Images 8799 saved.
Images 8800 saved.
Images 8801 saved.
Images 8802 saved.
Images 8803 saved.
Images 8804 saved.
Images 8805 saved.
Images 8806 saved.
Images 8807 saved.
Images 8808 saved.
Images 8809 saved.
Images 8810 saved.
Images 8811 saved.
Images 8812 saved.
Images 8813 

Images 9192 saved.
Images 9193 saved.
Images 9194 saved.
Images 9195 saved.
Images 9196 saved.
Images 9197 saved.
Images 9198 saved.
Images 9199 saved.
Images 9200 saved.
Images 9201 saved.
Images 9202 saved.
Images 9203 saved.
Images 9204 saved.
Images 9205 saved.
Images 9206 saved.
Images 9207 saved.
Images 9208 saved.
Images 9209 saved.
Images 9210 saved.
Images 9211 saved.
Images 9212 saved.
Images 9213 saved.
Images 9214 saved.
Images 9215 saved.
Images 9216 saved.
Images 9217 saved.
Images 9218 saved.
Images 9219 saved.
Images 9220 saved.
Images 9221 saved.
Images 9222 saved.
Images 9223 saved.
Images 9224 saved.
Images 9225 saved.
Images 9226 saved.
Images 9227 saved.
Images 9228 saved.
Images 9229 saved.
Images 9230 saved.
Images 9231 saved.
Images 9232 saved.
Images 9233 saved.
Images 9234 saved.
Images 9235 saved.
Images 9236 saved.
Images 9237 saved.
Images 9238 saved.
Images 9239 saved.
Images 9240 saved.
Images 9241 saved.
Images 9242 saved.
Images 9243 saved.
Images 9244 

Images 9629 saved.
Images 9630 saved.
Images 9631 saved.
Images 9632 saved.
Images 9633 saved.
Images 9634 saved.
Images 9635 saved.
Images 9636 saved.
Images 9637 saved.
Images 9638 saved.
Images 9639 saved.
Images 9640 saved.
Images 9641 saved.
Images 9642 saved.
Images 9643 saved.
Images 9644 saved.
Images 9645 saved.
Images 9646 saved.
Images 9647 saved.
Images 9648 saved.
Images 9649 saved.
Images 9650 saved.
Images 9651 saved.
Images 9652 saved.
Images 9653 saved.
Images 9654 saved.
Images 9655 saved.
Images 9656 saved.
Images 9657 saved.
Images 9658 saved.
Images 9659 saved.
Images 9660 saved.
Images 9661 saved.
Images 9662 saved.
Images 9663 saved.
Images 9664 saved.
Images 9665 saved.
Images 9666 saved.
Images 9667 saved.
Images 9668 saved.
Images 9669 saved.
Images 9670 saved.
Images 9671 saved.
Images 9672 saved.
Images 9673 saved.
Images 9674 saved.
Images 9675 saved.
Images 9676 saved.
Images 9677 saved.
Images 9678 saved.
Images 9679 saved.
Images 9680 saved.
Images 9681 